In [ ]:
import heapq

class LinkStateRouting:
    def __init__(self, nodes, graph):
        self.nodes = nodes
        self.graph = graph
        self.adjacency_list = {node: graph[node] for node in nodes}
        self.routing_table = {node: {} for node in nodes}
        self.shortest_paths = {}

    def dijkstra(self, source):
        pq = [(0, source)]
        visited = set()
        distances = {node: float('inf') for node in self.nodes}
        distances[source] = 0

        while pq:
            (dist, node) = heapq.heappop(pq)
            if node in visited:
                continue
            visited.add(node)

            for neighbor, cost in self.adjacency_list[node].items():
                new_dist = distances[node] + cost
                if new_dist < distances[neighbor]:
                    distances[neighbor] = new_dist
                    heapq.heappush(pq, (new_dist, neighbor))
                    self.shortest_paths[neighbor] = self.shortest_paths.get(node, []) + [neighbor]

        return distances

    def update_routing_table(self):
        for node in self.nodes:
            self.shortest_paths = {node: [node]}
            distances = self.dijkstra(node)
            for dest_node, cost in distances.items():
                if dest_node != node:
                    self.routing_table[node][dest_node] = (cost, self.shortest_paths[dest_node][1])

    def print_routing_table(self):
        print("Routing table:")
        for node in self.nodes:
            print(f"Node {node}:")
            for dest_node, (cost, next_hop) in self.routing_table[node].items():
                print(f"  {dest_node} -> {cost} via {next_hop}")

# Example usage
nodes = ['A', 'B', 'C', 'D']
graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'C': 2, 'D': 5},
    'C': {'A': 4, 'B': 2, 'D': 1},
    'D': {'B': 5, 'C': 1},
}
lsr = LinkStateRouting(nodes, graph)
lsr.update_routing_table()
lsr.print_routing_table()


Routing table:
Node A:
  B -> 1 via B
  C -> 3 via B
  D -> 4 via B
Node B:
  A -> 1 via A
  C -> 2 via C
  D -> 3 via C
Node C:
  A -> 3 via B
  B -> 2 via B
  D -> 1 via D
Node D:
  A -> 4 via C
  B -> 3 via C
  C -> 1 via C


In [ ]:
class DistanceVectorRouting:
    def __init__(self, nodes, graph):
        self.nodes = nodes
        self.graph = graph
        self.routing_table = {node: {dest_node: (cost, dest_node) for dest_node, cost in graph[node].items()} for node in nodes}
        self.neighbor_costs = {node: {neighbor: cost for neighbor, cost in graph[node].items()} for node in nodes}
        self.changed = True

    def update_routing_table(self):
        while self.changed:
            self.changed = False
            for node in self.nodes:
                for dest_node in self.graph[node]:
                    if dest_node == node:
                        continue
                    min_cost, next_hop = self.routing_table[node][dest_node]
                    for neighbor, cost in self.neighbor_costs[node].items():
                        if dest_node not in self.routing_table[neighbor]:
                            continue
                        new_cost = cost + self.routing_table[neighbor][dest_node][0]
                        if new_cost < min_cost:
                            min_cost, next_hop = new_cost, neighbor
                    if (min_cost, next_hop) != self.routing_table[node][dest_node]:
                        self.routing_table[node][dest_node] = (min_cost, next_hop)
                        self.changed = True

    def print_routing_table(self):
        print("Routing table:")
        for node in self.nodes:
            print(f"Node {node}:")
            for dest_node, (cost, next_hop) in self.routing_table[node].items():
                print(f"  {dest_node} -> {cost} via {next_hop}")

# Example usage
nodes = ['A', 'B', 'C', 'D']
graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'C': 2, 'D': 5},
    'C': {'A': 4, 'B': 2, 'D': 1},
    'D': {'B': 5, 'C': 1},
}
dvr = DistanceVectorRouting(nodes, graph)
dvr.update_routing_table()
dvr.print_routing_table()


Routing table:
Node A:
  B -> 1 via B
  C -> 3 via B
Node B:
  A -> 1 via A
  C -> 2 via C
  D -> 3 via C
Node C:
  A -> 3 via B
  B -> 2 via B
  D -> 1 via D
Node D:
  B -> 3 via C
  C -> 1 via C
